In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [2]:
# Hyperparameter
batch_size = 64
learning_rate = 0.001
num_epochs = 1

In [3]:

# Laden des FashionMNIST-Datensatzes
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = FashionMNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [4]:

# LeNet Modell Definition
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(2450, 120) #800
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2, stride=2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2, stride=2)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:

# Modellinitialisierung
model = LeNet()

# Verlustfunktion und Optimierer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:

# Trainingsschleife
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Vorwärtsdurchlauf
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Rückwärtsdurchlauf und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Schritt [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

# Testen des Modells
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Genauigkeit des Modells auf Testdaten: {} %'.format(100 * correct / total))


Epoch [1/1], Schritt [100/938], Loss: 0.7076
Epoch [1/1], Schritt [200/938], Loss: 0.4126
Epoch [1/1], Schritt [300/938], Loss: 0.5515
Epoch [1/1], Schritt [400/938], Loss: 0.4578
Epoch [1/1], Schritt [500/938], Loss: 0.5846
Epoch [1/1], Schritt [600/938], Loss: 0.3553
Epoch [1/1], Schritt [700/938], Loss: 0.5733
Epoch [1/1], Schritt [800/938], Loss: 0.3693
Epoch [1/1], Schritt [900/938], Loss: 0.2426
Genauigkeit des Modells auf Testdaten: 87.47 %
